In [10]:
import cv2
from paddleocr import PaddleOCR, draw_ocr
from matplotlib import pyplot as plt
import os

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/paddle/vision/transforms/functional_pil.py:36: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': Image.NEAREST,
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/paddle/vision/transforms/functional_pil.py:37: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': Image.BILINEAR,
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/paddle/vision/transforms/functional_pil.py:38: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': Image.BICUBIC,
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/paddle/vision/transforms/functional_pil.py:39: DeprecationWarning: BOX is deprecated and will be removed in Pillo

In [11]:
ocr_model = PaddleOCR(lang='en')

[2022/09/05 18:14:40] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, shape_info_filename=None, precision='fp32', gpu_mem=500, image_dir=None, det_algorithm='DB', det_model_dir='/home/studio-lab-user/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_box_type='quad', det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, det_fce_box_type='poly', rec_algorithm='SVTR_LCNet', rec_model_dir='/home/studio-lab-user/.paddleocr/whl/rec/en/en_PP-OCRv3_rec_infer', rec_image_shape='3, 48, 320', rec_bat

---    Fused 0 subgraphs into layer_norm op.
---    Fused 0 subgraphs into layer_norm op.


In [12]:
# To pull in previusly save data
import pandas as pd
data = pd.read_csv("image_relevant.csv", encoding='latin')
data.head()

,Unnamed: 0,id,description,image,images,relevant,processed_description,text_score
0,0,918707,Boys Among Us Hoodies Super Cool Print Clothes...,968bf5e6c99211c2ad8ac27fa55e4651b941d137,['968bf5e6c99211c2ad8ac27fa55e4651b941d137'],1,boys among us hoodies super cool print clothes...,0.192416
1,1,918708,Cat And Dog Cardboard Printing Jigsaw Puzzle M...,68ece3fab138feb537eee296f3348f3543e23a1d,"['68ece3fab138feb537eee296f3348f3543e23a1d', '...",0,cat dog cardboard printing jigsaw puzzle manuf...,0.030835
2,2,918709,0098 30 Pcs 21pack Sensory 4 Piece Puzzle Fidg...,a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf,"['a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf', '...",0,0098 30 pcs 21pack sensory 4 piece puzzle fidg...,0.001173
3,3,918710,Boys Sweatshirt Among us 3D Hoodies Cool Fashi...,a278812a832f24a822e2602b91a7b1f5749926cb,['a278812a832f24a822e2602b91a7b1f5749926cb'],1,boys sweatshirt among us 3d hoodies cool fashi...,0.275841
4,4,918711,20cm Among Us Soft Plush Toys Vocal Butt Toy S...,3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a,['3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a'],0,20cm among us soft plush toys vocal butt toy s...,0.000861


In [13]:
image_df = data[['image','relevant']]
image_df = image_df[:100]
len(image_df)

100

In [14]:
def get_words_and_scores_and_add_to_dataframe(row):
    photo = row["image"]
    if photo == "empty":
        return [[],[]]
    final = []
    image_path = f"/home/studio-lab-user/DS-Projects/OCR/SWOOP_OCR_images/{photo}.jpg"
    result = ocr_model.ocr(image_path)
    texts = [res[1][0] for res in result]
    # for word in texts:
    #     final.extend(word.split())
    scores = [res[1][1] *100 for res in result]
    result = [texts, scores]
    return result

In [15]:
image_df["words"] = image_df.apply(get_words_and_scores_and_add_to_dataframe, axis=1)

[2022/09/05 18:14:42] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2022/09/05 18:14:43] ppocr DEBUG: dt_boxes num : 2, elapse : 0.7650423049926758
[2022/09/05 18:14:43] ppocr DEBUG: rec_res num  : 2, elapse : 0.13434052467346191
[2022/09/05 18:14:43] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2022/09/05 18:14:44] ppocr DEBUG: dt_boxes num : 1, elapse : 0.7996869087219238
[2022/09/05 18:14:44] ppocr DEBUG: rec_res num  : 1, elapse : 0.09857559204101562
[2022/09/05 18:14:44] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2022/09/05 18:14:44] ppocr DEBUG: dt_boxes num : 0, elapse : 0.07112312316894531
[2022/09/05 18:14:44] ppocr DEBUG: rec_res num  : 0, elapse : 2.1457672119140625e-06
[2022/09/05 18:14:44] ppocr WARNING: Since the angle classif

In [17]:
import re
import string

def clean_text(row):
    word_list = row["words"][0]
    new_word_list = []
    for word in word_list:
        new_word = word.lower()
        new_word = re.sub("[%s]" % re.escape(string.punctuation), " ", new_word)
        new_word_list.append(new_word)
    return [new_word_list, row["words"][1]]

In [18]:
image_df["words"] = image_df.apply(clean_text, axis=1)

In [20]:
image_df.head(5)

,image,relevant,words
0,968bf5e6c99211c2ad8ac27fa55e4651b941d137,1,"[[amongus], [82.79706239700317]]"
1,68ece3fab138feb537eee296f3348f3543e23a1d,0,"[[zfsz en alibaba com], [92.13441610336304]]"
2,a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf,0,"[[], []]"
3,a278812a832f24a822e2602b91a7b1f5749926cb,1,"[[impostor], [97.47896194458008]]"
4,3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a,0,"[[press to make a sound], [99.68701601028442]]"


In [21]:
## define the target words / phrases

targets = ["among us", "imposter"]

In [23]:
import pandas as pd
import io
import matplotlib.pyplot as plt
import time
from sklearn.metrics import roc_auc_score

import re
import string
import nltk
import itertools
import numpy as np

# this allows you to apply a progress bar to pandas operations
from tqdm import tqdm
tqdm.pandas()

In [ ]:
# helper functions for funal scoring function

def advance(iterator, step):
    next(itertools.islice(iterator, step, step), None)

def tuplewize(iterable, size):
    iterators = itertools.tee(iterable, size)
    for position, iterator in enumerate(iterators):
        advance(iterator, position)
    return zip(*iterators)

In [36]:
sample_list = image_df["words"][6][0]
score_list = image_df["words"][6][1]
max_score = 0

for target in targets:
    phrase_length = len(target.split())
    for c, s in zip(tuplewize(sample_list, phrase_length), tuplewize(score_list, phrase_length)):
        # print("c:", c)
        new_sample = " ".join(c)
        print("new_sample:", new_sample)
        edit_distance = nltk.edit_distance(target, new_sample)
        print("target:", target)
        print("edit_distance:",edit_distance)
        closeness = (len(target)-edit_distance)/len(target)
        confidence_score = np.mean(s)
        final_score = closeness * confidence_score
        if final_score > max_score:
            max_score = final_score
            
    print(max_score/100)

new_sample: among us among
target: among us
edit_distance: 6
new_sample: among amonc us
target: among us
edit_distance: 6
new_sample: amonc us you
target: among us
edit_distance: 5
new_sample: you you
target: among us
edit_distance: 7
new_sample: you look
target: among us
edit_distance: 8
new_sample: look lggk
target: among us
edit_distance: 8
new_sample: lggk sus
target: among us
edit_distance: 6
new_sample: sus mb
target: among us
edit_distance: 7
new_sample: mb trust
target: among us
edit_distance: 6
0.3311057277023792
new_sample: among us
target: imposter
edit_distance: 7
new_sample: among
target: imposter
edit_distance: 6
new_sample: amonc us
target: imposter
edit_distance: 7
new_sample: you
target: imposter
edit_distance: 7
new_sample: you
target: imposter
edit_distance: 7
new_sample: look
target: imposter
edit_distance: 7
new_sample: lggk
target: imposter
edit_distance: 8
new_sample: sus
target: imposter
edit_distance: 7
new_sample: mb
target: imposter
edit_distance: 7
new_sampl